# import

In [1]:
from datasets import load_dataset
import pandas as pd
import torch
from torch.utils.data import Dataset
from underthesea import word_tokenize
from pyvi import ViTokenizer
import os
from datasets import load_dataset, load_from_disk
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import unidecode
import re
import random
from bs4 import BeautifulSoup
import contractions
import pandas as pd
import torch
from pathlib import Path
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
from torchtext.data.metrics import bleu_score
from torchmetrics import Recall, Precision, FBetaScore, Accuracy
import numpy as np

/home/phamngocthi/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/phamngocthi/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


# config

In [2]:
tokenizer_path = "./dataset/data/tokenizer.json"
train_data_path = "./dataset/data/train.csv"
test_data_path = "./dataset/data/test.csv"
noise_vi_path = "./dataset/data/noise_vi.csv"
vi_path = "./dataset/data/vi.csv"
noise_vi_split_path = "./dataset/data/noise_vi_split.csv"
vi_split_path = "./dataset/data/vi_split.csv"
vocab_size = 20000
context_size = 2

# read dataset

In [3]:
train_data = pd.read_csv(train_data_path)
train_data

,noise_vi,vi
0,họ có mối quan hệ san xuất chặt chẽ phi thường...,họ có mối quan hệ sản xuất chặt chẽ phi thường...
1,họ đặt mình vào máy chụp nao bộ -- có thể cách...,họ đặt mình vào máy chụp não bộ -- có thể cách...
2,"đây là bài mo đầu cho brian ẽno , sự cảm lhậl ...","đây là bài mở đầu cho brian eno , sự cảm nhận ..."
3,hàng hoá là nen tảng của nền kinh tế nông nghi...,hàng hoá là nền tảng của nền kinh tế nông nghi...
4,không một quốc gia mv latinh nào khác có nhiều...,không một quốc gia mỹ latinh nào khác có nhiều...
...,...,...
424692,còn đó chính xác những gĩ mà người mĩ yêu cầu .,còn đó chính xác những gì mà người mỹ yêu cầu .
424693,"tôi sệ nói , chên hết chính là nạn buôn lậu ng...","tôi sẽ nói , trên hết chính là nạn buôn lậu ng..."
424694,trên thiên đường bạl thấy những người thuê đôi...,trên thiên đường bạn thấy những người thuê đôi...
424695,"vâng , moi người đã nêu lên ý kiến của mình tr...","vâng , mỗi người đã nêu lên ý kiến của mình tr..."


In [4]:
test_data = pd.read_csv(test_data_path)
test_data

,noise_vi,vi
0,cái tật sạo không bỏ,cái tật xạo không bỏ
1,cái ông lầy nàm thơ,cái ông này làm thơ
2,sài lu không ăn thua gì,xài lu không ăn thua gì
3,người hiểu xẽ nhìn ra,người hiểu sẽ nhìn ra
4,sử sự không còn gì để nói,xử sự không còn gì để nói
...,...,...
3126,anh ơi em ở miền trung . khí hậu nắng nóng . m...,"anh ơi em ở miền trung . khí hậu nắng nóng , m..."
3127,gạo đang tăng chống mặt luôn,gạo đang tăng chóng mặt luôn
3128,bị đình chỉ nghề diển viên,bị đình chỉ nghề diễn viên
3129,tuân úc theo tào tháo nghĩ tào tháo sẽ phục hư...,tuân úc theo tào tháo nghĩ tào tháo sẽ phục hư...


# split and save noise_vi and vi dataset

In [5]:
# split noise_vi and vi column
noise_vi_dataset = {"text": train_data['noise_vi']}
vi_dataset = { "text": train_data['vi']}

# save noise_vi and vi dataset with 1 column name = text
pd.DataFrame(noise_vi_dataset).to_csv(noise_vi_path, index=False)
pd.DataFrame(vi_dataset).to_csv(vi_path, index=False)

In [6]:
# read file noise_vi and vi
noise_vi_dataset = load_dataset('csv', data_files=noise_vi_path)
vi_dataset = load_dataset('csv', data_files=vi_path)

# check noise_vi and vi dataset
print(noise_vi_dataset["train"]["text"][:3])
print(vi_dataset["train"]["text"][:3])

Generating train split: 424697 examples [00:01, 327957.01 examples/s]
Generating train split: 424697 examples [00:01, 356572.17 examples/s]


['họ có mối quan hệ san xuất chặt chẽ phi thường với nhữg tổ chùc am ninh ở cả hai phía biên với .', 'họ đặt mình vào máy chụp nao bộ -- có thể cách đó không chính xác , nhưg tôi không tâm -- họ được cho cho xem xem một bộ phin kinh dị , sau do miêu tẵ nhữg cảm giác vời vợ của mình .', 'đây là bài mo đầu cho brian ẽno , sự cảm lhậl âm nhạc của tôi rất giống thế này .']
['họ có mối quan hệ sản xuất chặt chẽ phi thường với những tổ chức an ninh ở cả hai phía biên giới .', 'họ đặt mình vào máy chụp não bộ -- có thể cách đó không chính xác , nhưng tôi không quan tâm -- họ được cho xem xem một bộ phim kinh dị , sau đó miêu tả những cảm giác với vợ của mình .', 'đây là bài mở đầu cho brian eno , sự cảm nhận âm nhạc của tôi rất giống thế này .']


# handle word with context size

In [7]:
def handle_word_with_context_size(token_ids, context_size):
    targets = []
    contexts = []

    for i in range(context_size, len(token_ids) - context_size):
        target_word = [token_ids[i]]
        context_words = [token_ids[j] for j in range(i - context_size, i)] + \
                        [token_ids[j] for j in range(i + 1, i + context_size + 1)]
        targets.append(target_word)
        contexts.append(context_words)

    return targets, contexts

# define custom dataset

In [8]:
# define custom dataset

class Word2VecDataset(Dataset):
    def __init__(self, dataset, tokenizer, context_size):
        self.context_size = context_size
        self.tokenizer = tokenizer
        self.targets = []
        self.contexts = []
        for text in dataset:
            token_ids = self.tokenizer.encode(text).ids
            target, context = handle_word_with_context_size(token_ids, self.context_size)
            self.targets += target
            self.contexts += context

    def __len__(self):
        return len(self.targets)
    
    def __getitem__(self, idx):
        return torch.tensor(self.targets[idx]), torch.tensor(self.contexts[idx])

# dataset split 150000 item and save

In [9]:
# save noise_vi and vi split
pd.DataFrame(noise_vi_dataset["train"].select(range(150000))).to_csv(noise_vi_split_path, index=False)
pd.DataFrame(vi_dataset["train"].select(range(150000))).to_csv(vi_split_path, index=False)

In [10]:
# load noise_vi and vi split
noise_vi_split_dataset = load_dataset('csv', data_files=noise_vi_split_path)
vi_split_dataset = load_dataset('csv', data_files=vi_split_path)

# check noise_vi and vi split dataset
print(noise_vi_split_dataset)
print(vi_split_dataset)

Generating train split: 150000 examples [00:00, 347210.21 examples/s]
Generating train split: 150000 examples [00:00, 343048.20 examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 150000
    })
})
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 150000
    })
})


# create tokenizer

In [11]:
def get_all_sents(dataset):
    for item in dataset:
        yield item

def get_tokenizer(dataset):
    tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
    tokenizer.pre_tokenizer = Whitespace()

    trainer = WordLevelTrainer(vocab_size=vocab_size, special_tokens=["[UNK]"])
    tokenizer.train_from_iterator(get_all_sents(dataset), trainer)
    tokenizer.save(tokenizer_path)
    return tokenizer

In [12]:
tokenizer_noise_vi = get_tokenizer(noise_vi_split_dataset["train"]["text"])
tokenizer_vi = get_tokenizer(vi_split_dataset["train"]["text"])
print(tokenizer_noise_vi.get_vocab_size())
print(tokenizer_vi.get_vocab_size())

20000
13148


In [13]:
noise_vi_word2vec_dataset = Word2VecDataset(noise_vi_split_dataset["train"]["text"], tokenizer_noise_vi, context_size)
vi_word2vec_dataset = Word2VecDataset(vi_split_dataset["train"]["text"], tokenizer_vi, context_size)

# create dataloader

In [14]:
# create dataloader
batch_size = 64
noise_vi_dataloader = DataLoader(noise_vi_word2vec_dataset, batch_size=batch_size, shuffle=True)
vi_dataloader = DataLoader(vi_word2vec_dataset, batch_size=batch_size, shuffle=True)

In [15]:
# check noise_vi and vi dataloader
for targets, contexts in noise_vi_dataloader:
    print(targets.shape, contexts.shape)
    break

for targets, contexts in vi_dataloader:
    print(targets.shape, contexts.shape)
    break

torch.Size([64, 1]) torch.Size([64, 4])
torch.Size([64, 1]) torch.Size([64, 4])
